###### CELLO Mk 3.0, EST. 2023-3-3


## COMPONENT #2:
## Restricted Graph Conversion to Python netlist object

SAVES the Netlist JSON file produced by YOSYS as an OBJECT such that we can map and evaluates parts to it

In [1]:
import json

In [2]:
out_path = '../../IO/YOSYS'

In [3]:
# ENTRY POINT
netlist_name = 'md5_opt'

In [4]:
netlist_path = out_path + '/' + netlist_name + '.json'

In [5]:
netlist_file = open(netlist_path, 'r')
netlist = json.load(netlist_file)

# important: netlist has in the json format generated by YOSYS
# TODO: Add conversion from other netlist types (such as STL)
print(len(netlist))
print(type(netlist))
print(netlist.keys())
print(netlist['modules'].keys())
print(netlist['modules']['md5Core'].keys())
for k in netlist['modules']['md5Core'].keys():
    print(k)
    print(netlist['modules']['md5Core'][k].keys())

2
<class 'dict'>
dict_keys(['creator', 'modules'])
dict_keys(['md5Core'])
dict_keys(['attributes', 'parameter_default_values', 'ports', 'cells', 'netnames'])
attributes
dict_keys(['hdlname', 'dynports', 'top', 'src'])
parameter_default_values
dict_keys(['WIDTH'])
ports
dict_keys(['a_i', 'b_i', 'c_i', 'd_i', 'round_i', 'm_i', 's_i', 't_i', 'a_o'])
cells
dict_keys(['$auto$opt_expr.cc:607:replace_const_cells$957', '$auto$opt_expr.cc:607:replace_const_cells$959', '$auto$simplemap.cc:126:simplemap_reduce$361', '$auto$simplemap.cc:126:simplemap_reduce$371', '$auto$simplemap.cc:126:simplemap_reduce$381', '$auto$simplemap.cc:126:simplemap_reduce$395', '$auto$simplemap.cc:126:simplemap_reduce$397', '$auto$simplemap.cc:126:simplemap_reduce$399', '$auto$simplemap.cc:126:simplemap_reduce$401', '$auto$simplemap.cc:126:simplemap_reduce$411', '$auto$simplemap.cc:126:simplemap_reduce$413', '$auto$simplemap.cc:196:simplemap_lognot$363', '$auto$simplemap.cc:196:simplemap_lognot$373', '$auto$simplemap.cc

In [6]:
# test print the entire netlist
print(json.dumps(netlist, indent=4))

{
    "creator": "Yosys 0.27 (git sha1 5f88c218b58, clang 14.0.0 -fPIC -Os)",
    "modules": {
        "md5Core": {
            "attributes": {
                "hdlname": "\\md5Core",
                "dynports": "00000000000000000000000000000001",
                "top": "00000000000000000000000000000001",
                "src": "md5Core.v:39.1-107.10"
            },
            "parameter_default_values": {
                "WIDTH": "00000000000000000000000000000010"
            },
            "ports": {
                "a_i": {
                    "direction": "input",
                    "upto": 1,
                    "bits": [
                        2,
                        3
                    ]
                },
                "b_i": {
                    "direction": "input",
                    "upto": 1,
                    "bits": [
                        4,
                        5
                    ]
                },
                "c_i": {
                    "d

In [7]:
# print(len(netlist))
# # 2 is right
circuit_name = list(netlist["modules"].keys())[0]
print("NAME: " + str(circuit_name))

NAME: md5Core


In [8]:
components = []
# dev res/s
for m in netlist["modules"][circuit_name]:
    components.append(m)
    print(m)
print()

if 'parameter_default_values' in components:
    print("we've got an extra component here! \n")
    print(json.dumps(netlist['modules'][circuit_name]['parameter_default_values'], indent=4))
else:
    print('we\'re good')

attributes
parameter_default_values
ports
cells
netnames

we've got an extra component here! 

{
    "WIDTH": "00000000000000000000000000000010"
}


In [9]:
# temporarily define the componenets of the netlist as vars
ports = netlist["modules"][circuit_name]['ports']
cls = netlist["modules"][circuit_name]['cells']
edges = netlist["modules"][circuit_name]['netnames']

# scan for special parameters and attributes
for gatename, gatedata in cls.items():
    if gatedata['parameters']:
        print(f"GATE '{gatename}' has PARAMETER '{gatedata['parameters']}'")
    elif gatedata['attributes']:
        print(f"GATE '{gatename}' has ATTRIBUTE '{gatedata['attributes']}'")

for netname, netdata in edges.items():
    if 'src' in netdata['attributes']:
        print(f"NET '{netname}' has SOURCE '{netdata['attributes']['src']}'")

GATE '$auto$simplemap.cc:126:simplemap_reduce$361' has ATTRIBUTE '{'src': 'md5Core.v:0.0-0.0|md5Core.v:87.3-104.10'}'
GATE '$auto$simplemap.cc:126:simplemap_reduce$371' has ATTRIBUTE '{'src': 'md5Core.v:0.0-0.0|md5Core.v:87.3-104.10'}'
GATE '$auto$simplemap.cc:126:simplemap_reduce$381' has ATTRIBUTE '{'src': 'md5Core.v:0.0-0.0|md5Core.v:87.3-104.10'}'
GATE '$auto$simplemap.cc:126:simplemap_reduce$395' has ATTRIBUTE '{'src': 'md5Core.v:0.0-0.0|md5Core.v:87.3-104.10|/opt/homebrew/bin/../share/yosys/techmap.v:593.20-593.31'}'
GATE '$auto$simplemap.cc:126:simplemap_reduce$397' has ATTRIBUTE '{'src': 'md5Core.v:0.0-0.0|md5Core.v:87.3-104.10|/opt/homebrew/bin/../share/yosys/techmap.v:593.20-593.31'}'
GATE '$auto$simplemap.cc:126:simplemap_reduce$399' has ATTRIBUTE '{'src': 'md5Core.v:0.0-0.0|md5Core.v:87.3-104.10|/opt/homebrew/bin/../share/yosys/techmap.v:593.20-593.31'}'
GATE '$auto$simplemap.cc:126:simplemap_reduce$401' has ATTRIBUTE '{'src': 'md5Core.v:0.0-0.0|md5Core.v:87.3-104.10|/opt/h

In [10]:
'''
_sort_gates(self, cells)

'''

for c in cls.keys():
    partition = list(c.split('$'))
    gate_id = partition[-1]
    gate_type = cls[c]['type']
#     print(gate_id + ' ' + gate_type + ' ' + partition[1])
    gate = {gate_id : 
            {
             'type' : gate_type,
             'inputs' : {},
             'output' : {},
            }
           }
#     print(gate)
#     print(json.dumps(gate, indent=4))
#     print()
    dirns = cls[c]['port_directions']
    ctns = cls[c]['connections']
    inputs = []
    outputs = []
    for d in dirns.keys():
        inout = dirns[d]
        if inout == 'input':
            inputs.append(d)
        else:
            outputs.append(d)
    gate_inputs = []
    gate_outputs = []
    for c in ctns.keys():
        c_nodes = ctns[c]
        if c in inputs:
            gate_inputs.append((c, c_nodes))
        else:
            gate_outputs.append((c, c_nodes))
#     gate[gate_id]['inputs'] = gate_inputs
#     gate[gate_id]['output'] = gate_outputs
#     print(gate_inputs)
#     print(gate_outputs)
    for tup in gate_inputs:
        (node_name, edge_nos) = tup
        try:
            gate[gate_id]['inputs'][node_name] += edge_nos
        except Exception as e:
            gate[gate_id]['inputs'][node_name] = edge_nos
            # print('creating node: ' + str(e))
    for tup in gate_outputs:
        (node_name, edge_nos) = tup
        try:
            gate[gate_id]['output'][node_name] += edge_nos
        except Exception as e:
            gate[gate_id]['output'][node_name] = edge_nos
            # print('creating node: ' + str(e))
    # print(json.dumps(gate, indent=4))
#         print(d + ' ' + str(dirns[d]))
#     for n in ctns.keys():
print("pass")

pass


In [11]:


in_nodes = []
out_nodes = []
for p in ports.keys():
    node_name = p
    direction = ports[p]['direction']
    bits = ports[p]['bits']
    node = (node_name, bits)
    if direction == 'input':
        in_nodes.append(node)
    elif direction == 'output':
        out_nodes.append(node)
    else:
        raise ValueError('Invalid [in/out]put node') 
print("inputs: \n" + str(in_nodes) + '\n')
print("outputs: \n" + str(out_nodes))

inputs: 
[('a_i', [2, 3]), ('b_i', [4, 5]), ('c_i', [6, 7]), ('d_i', [8, 9]), ('round_i', [10, 11]), ('m_i', [12, 13]), ('s_i', [14, 15]), ('t_i', [16, 17])]

outputs: 
[('a_o', [18, 19])]


In [12]:
class Netlist:
    def __init__(self, netlistJSON):
        net_main = netlistJSON['modules']
        self.name = list(net_main.keys())[0]
        ports = net_main[self.name]['ports']
        cells = net_main[self.name]['cells']
        edges = net_main[self.name]['netnames']
        i, o = self.__sort_nodes(ports)
        self.inputs = i
        self.outputs = o
        self.gates = self.__sort_gates(cells)
        self.edges = self.__check_edges(edges)
        
    def __sort_nodes(self, ports):
        in_nodes = []
        out_nodes = []
        for p in ports.keys():
            node_name = p
            direction = ports[p]['direction']
            bits = ports[p]['bits']
            node = (node_name, bits)
            if direction == 'input':
                in_nodes.append(node)
            elif direction == 'output':
                out_nodes.append(node)
            else:
                raise ValueError('Invalid [in/out]put node') 
        return (in_nodes, out_nodes)
    
    def __sort_gates(self, cells):
        gates = {}
        for c in cells.keys():
            partition = list(c.split('$'))
            gate_id = partition[-1]
            gate_type = cells[c]['type']
            gate = {gate_id : 
                    {
                     'type' : gate_type,
                     'inputs' : {},
                     'output' : {},
                    }
                   }
            dirns = cells[c]['port_directions']
            ctns = cells[c]['connections']
            inputs = []
            outputs = []
            for d in dirns.keys():
                inout = dirns[d]
                if inout == 'input':
                    inputs.append(d)
                else:
                    outputs.append(d)
            gate_inputs = []
            gate_outputs = []
            for c in ctns.keys():
                c_nodes = ctns[c]
                if c in inputs:
                    gate_inputs.append((c, c_nodes))
                else:
                    gate_outputs.append((c, c_nodes))
            for tup in gate_inputs:
                (node_name, edge_nos) = tup
                try:
                    gate[gate_id]['inputs'][node_name] += edge_nos
                except Exception as e:
                    gate[gate_id]['inputs'][node_name] = edge_nos
            for tup in gate_outputs:
                (node_name, edge_nos) = tup
                try:
                    gate[gate_id]['output'][node_name] += edge_nos
                except Exception as e:
                    gate[gate_id]['output'][node_name] = edge_nos
            gates.update(gate)
        return gates
    
    def __str__(self):
        return (f"{self.name}, with: \n"
                f"{len(self.inputs)} inputs,\n"
                f"{len(self.outputs)} ouputs.")
    
    def __check_edges(self, edges):
        for n in edges:
            pass
        return 9

In [13]:
testNet = Netlist(netlist)

In [14]:
print(testNet.inputs)

[('a_i', [2, 3]), ('b_i', [4, 5]), ('c_i', [6, 7]), ('d_i', [8, 9]), ('round_i', [10, 11]), ('m_i', [12, 13]), ('s_i', [14, 15]), ('t_i', [16, 17])]


In [15]:
print(testNet.outputs)

[('a_o', [18, 19])]


In [16]:
print(json.dumps(testNet.gates, indent=4))

{
    "957": {
        "type": "$_NOT_",
        "inputs": {
            "A": [
                10
            ]
        },
        "output": {
            "Y": [
                20
            ]
        }
    },
    "959": {
        "type": "$_NOT_",
        "inputs": {
            "A": [
                11
            ]
        },
        "output": {
            "Y": [
                21
            ]
        }
    },
    "361": {
        "type": "$_OR_",
        "inputs": {
            "A": [
                20
            ],
            "B": [
                21
            ]
        },
        "output": {
            "Y": [
                22
            ]
        }
    },
    "371": {
        "type": "$_OR_",
        "inputs": {
            "A": [
                10
            ],
            "B": [
                21
            ]
        },
        "output": {
            "Y": [
                23
            ]
        }
    },
    "381": {
        "type": "$_OR_",
        "inp

In [17]:
print(testNet)

md5Core, with: 
8 inputs,
1 ouputs.
